In [ ]:
import numpy as np
import pandas as pd

from numpy.linalg import inv

# Loading Data_Frame
data_x_df = pd.read_csv('data_X.csv')
data_t_df = pd.read_csv('data_T.csv')
data_df = pd.merge(data_x_df, data_t_df, left_index=True, right_index=True)

sizeData = data_df.shape[0]
sizeFeature = data_df.shape[1]

# Create Dataset
rawData = np.array(data_df)

# Random Shuffle for Raw Data
np.random.shuffle(rawData)

# Split Dataset into Train & Validation
trainData = rawData[:int(sizeData*90/100)]
valData = rawData[int((-sizeData)*10/100):]

sizeTrain = trainData.shape[0]
sizeVal = valData.shape[0]

In [ ]:
"""
Function:
- Create Phi(x) for M=1 & M=2

Parameters:
- train: trainData[:, :-1]
"""
def createPhi(train):
    len = train.shape[1]
    
    # M >= 1
    phiOne = np.insert(train, 0, values=np.ones(train.shape[0]), axis=1)
    
    # M >= 2
    phiTwo = phiOne
    diff = 0
    offset = len + 1;
    for i in range(len):
        for j in range(len):
            if i <= j:
                offset += 1
                phiTwo = np.insert(phiTwo, 
                                  i*len + j + len + 1 - diff,
                                  values = train[:, i]*train[:, j],
                                  axis = 1)
            else:
                diff += 1
    # M >= 3
    diff = 0
    phiThree = phiTwo
    for i in range(len):
        for j in range(len):
            for k in range(len):
                if i <= j and j <= k:
                    phiThree = np.insert(phiThree,
                                        i*len*len + j*len + k + offset - diff,
                                        values = train[:, i]*train[:, j]*train[:, k],
                                        axis = 1)
                else:
                    diff += 1

    return phiOne, phiTwo, phiThree

In [ ]:
"""
Function:
- Create y(x,w) as Prediction function

Parameters:
- inputX : trainData[:, :-1]
- inputT : trainData[:, -1]

Return:
- Root Mean Square Value
"""
def prediction(inputX, inputT, M, weight):
    len = inputX.shape[1] #8
    y = np.zeros(inputX.shape[0])
    y += weight[0]
    
    # M >= 1 
    for i in range(len):
        m1 = np.multiply(inputX[:, i], weight[i+1])
        y += m1
    
    # M >= 2 with interaction terms
    offset = len + 1;
    if M >= 2:
        diff = 0
        for i in range(len):
            for j in range(len):
                if i <= j:
                    offset += 1
                    m2 = np.multiply(inputX[:, i], inputX[:, j])
                    y += np.multiply(m2, weight[i*len + j + len + 1 - diff])
                else:
                    diff += 1
    
    # M >= 3 with interaction terms (for (2-b))
    if M >= 3:
        diff = 0
        for i in range(len):
          for j in range(len):
              for k in range(len):
                  if i <= j and j <= k:
                      m3 = np.multiply(np.multiply(inputX[:, i], inputX[:, j]), inputX[:, k])
                      y += np.multiply(m3, weight[i*len*len + j*len + k + offset - diff])                    
                  else:
                      diff += 1
    
    return np.sqrt(np.mean((y - inputT)**2)) 

In [ ]:
# Feature Selection - (a)
phiOne, phiTwo, _ = createPhi(trainData[..., :-1])

# perform Least Square solution
"""
(X.T*X)-1 * X.T * Y
"""
weightM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiOne.T, phiOne)),
                      phiOne.T
                  ),
                  trainData[:, -1]) # Target
weightM2 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiTwo.T, phiTwo)),
                      phiTwo.T
                  ),
                  trainData[:, -1]) # Target

train_MLE_RMS = [prediction(trainData[..., :-1], trainData[..., -1], 1, weightM1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, weightM2)]
val_MLE_RMS = [prediction(valData[..., :-1], valData[..., -1], 1, weightM1), 
            prediction(valData[..., :-1], valData[..., -1], 2, weightM2)]


In [ ]:
print("===Feature Selection - (a)===\n")
print("Weight : \n")
print("M = 1 : \n", weightM1)
print("M = 2 : \n", weightM2, '\n')

print("Trainning set's RMS result:")
print("M = 1  ", train_MLE_RMS[0])
print("M = 2  ", train_MLE_RMS[1], '\n')

print("Valid set's RMS result:")
print("M = 1  ", val_MLE_RMS[0])
print("M = 2  ", val_MLE_RMS[1], '\n')

===Feature Selection - (a)===

Weight : 

M = 1 : 
 [-3.59038425e+06 -4.27490817e+04 -4.24587719e+04  1.14458276e+03
 -8.97466986e+00  1.21289581e+02 -3.70838142e+01  4.05913716e+01
  4.05740738e+04]
M = 2 : 
 [ 5.84170084e-01 -1.07969190e+02  2.29680500e+01  2.74211861e+01
 -7.62585815e+02 -5.45923221e+00 -4.97846699e+01  3.15244688e+01
  1.33210089e+01  7.70624180e+01  4.76441335e+02 -2.47029109e+01
 -8.17814970e+00 -3.02303669e+00  1.29117465e+00 -3.57380119e+00
 -5.77907483e+02  7.16943786e+02 -1.18167019e+02 -7.39491696e+00
  1.59214867e+00  6.38893876e+00 -1.99329922e+01 -3.62579950e+02
  2.32368726e+01 -5.78737586e-01  2.23877494e+00 -1.68933619e+00
  5.70376134e+00  2.83506664e+02 -1.62784095e-03  4.18018943e-02
 -4.84010368e-03 -3.37399914e-03  7.58279995e+00 -1.90020170e-01
  1.42695264e-02  6.66846648e-02 -4.38814142e+01  3.44666963e-03
 -1.51935879e-02 -7.09171924e+00  1.13700104e-02  2.97212345e+01
 -2.12956359e+03] 

Trainning set's RMS result:
M = 1   69335.8421124908
M 

In [ ]:
# Feature Selection - (b)

deleteOneOfFeature = [np.delete(trainData, i, 1) for i in range(trainData.shape[1]-1)]

phiOneArray = []
for i in range(trainData.shape[1]-1):
    _phiOne, _ , _= createPhi(deleteOneOfFeature[i][...,:-1])
    phiOneArray.append(_phiOne)

weightMLE1Array = []
for i in range(trainData.shape[1]-1):
    _weightM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiOneArray[i].T, phiOneArray[i])),
                      phiOneArray[i].T
                  ),
                  deleteOneOfFeature[i][..., -1]) # Target
    weightMLE1Array.append(_weightM1)          
      
train_MLE_RMS_Array = []
for i in range(trainData.shape[1]-1):
    _trainRMS = prediction(deleteOneOfFeature[i][..., :-1], deleteOneOfFeature[i][..., -1], 1, weightMLE1Array[i])
    train_MLE_RMS_Array.append(_trainRMS)

In [ ]:
print("===Feature Selection - (b)===\n")

for i in range(len(train_MLE_RMS_Array)):
    print("Delete No.", i+1, "Feature => RMS would be : ",train_MLE_RMS_Array[i])

print("\n\nWe can find that the final RMS is the most,")
print("which means without the median_income, the final predict would be the most unreliable.\n")
print("[Median_income] is the most contributive feature!!")

===Feature Selection - (b)===

Delete No. 1 Feature => RMS would be :  75200.43105183268
Delete No. 2 Feature => RMS would be :  75792.25786321914
Delete No. 3 Feature => RMS would be :  70513.95787274062
Delete No. 4 Feature => RMS would be :  69557.36171825083
Delete No. 5 Feature => RMS would be :  69851.35337406957
Delete No. 6 Feature => RMS would be :  71374.24408517867
Delete No. 7 Feature => RMS would be :  69384.96884794964
Delete No. 8 Feature => RMS would be :  90785.19250493227


We can find that the final RMS is the most,
which means without the median_income, the final predict would be the most unreliable.

[Median_income] is the most contributive feature!!


In [ ]:
# Maximum likelihood approach - (a)
print("===Maximum likelihood approach - (a)===\n")
print('Choose [Polynomial] as the basis function!!')
print('Polynomial models have moderate flexibility of shapes.')
print('Polynomial models have well known and understood properties')

===Maximum likelihood approach - (a)===

Choose [Polynomial] as the basis function!!
Polynomial models have moderate flexibility of shapes.
Polynomial models have well known and understood properties


In [ ]:
"""
Function:
- Gauss-Jordan 

Parameters:
- matrix

Return:
- inverse matrix
"""

def Gauss_Jordan(M):
    # Applying Gauss Jordan Elimination
    I = np.eye(M.shape[0])
    inv = np.linalg.solve(M, I)
    return inv

In [ ]:
# Maximum likelihood approach - (b)

_, _, phiThree = createPhi(trainData[..., :-1])

# perform Least Square solution
# """
# W = (X.T*X)^-1 * X.T * Y
# """

_inv = Gauss_Jordan(np.dot(phiThree.T, phiThree))
weightM3 = np.dot(np.dot(
                      _inv,
                      phiThree.T),
                  trainData[:, -1]) # Target

_train_MLE_RMS = prediction(trainData[..., :-1], trainData[..., -1], 3, weightM3)
_val_MLE_RMS = prediction(valData[..., :-1], valData[..., -1], 3, weightM3)

In [ ]:
# Maximum likelihood approach - (b)
print("===Maximum likelihood approach - (b)===\n")

print('As polynomial model become higher degree, the curve may oscillate between exact-fit values.')
print('For example below, Setting M = 3 is much more complicated for this case,')
print('Since we found that valid set RMS is from the Earth to the Moon which means model is overfitting.\n')

print("Trainning set's RMS result:")
print("M = 1  ", train_MLE_RMS[0])
print("M = 2  ", train_MLE_RMS[1])
print("M = 3  ", _train_MLE_RMS, '\n')

print("Valid set's RMS result:")
print("M = 1  ", val_MLE_RMS[0])
print("M = 2  ", val_MLE_RMS[1])
print("M = 3  ", _val_MLE_RMS, '\n')



===Maximum likelihood approach - (b)===

As polynomial model become higher degree, the curve may oscillate between exact-fit values.
For example below, Setting M = 3 is much more complicated for this case,
Since we found that valid set RMS is from the Earth to the Moon which means model is overfitting.

Trainning set's RMS result:
M = 1   69335.8421124908
M = 2   66820.92212048505
M = 3   58426.87588212439 

Valid set's RMS result:
M = 1   71571.57181339637
M = 2   68432.64200562023
M = 3   60339.4505880152 



In [ ]:
# Maximum likelihood approach - (c)

# Apply N-fold cross-validation
N_Fold = 100
sizeFold = int(trainData.shape[0]/N_Fold)

for n in range(N_Fold):
    _val = trainData[sizeFold*n:sizeFold*(n+1), :]
    _train = np.concatenate((trainData[0:sizeFold*n, :], trainData[sizeFold*(n+1):, :]))

    _phiOne, _phiTwo, _ = createPhi(_train[..., :-1])

    _wM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(_phiOne.T, _phiOne)),
                     _phiOne.T
                  ),
                  _train[:, -1]) # Target
    _wM2 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(_phiTwo.T, _phiTwo)),
                      _phiTwo.T
                  ),
                  _train[:, -1]) # Target
    if n==0:
        _weightM1 = _wM1
        _weightM2 = _wM2
    else:
        _weightM1 += _wM1
        _weightM2 += _wM2
    
_weightM1 = _weightM1/N_Fold
_weightM2 = _weightM2/N_Fold

train_MLE_RMS_N_Fold = [prediction(trainData[..., :-1], trainData[..., -1], 1, _weightM1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, _weightM2)]
val_MLE_RMS_N_Fold = [prediction(valData[..., :-1], valData[..., -1], 1, _weightM1), 
            prediction(valData[..., :-1], valData[..., -1], 2, _weightM2)]

In [ ]:
print("===Maximum likelihood approach - (c)===\n")
print('Selecting two different order to compare the behavior of model when performing N-Fold cross validation,')
print('We can found when test validation set in M = 2, the RMS tilts up,')
print('We can conclude that M = 2 may be too much complex for this scenario,')
print('Since its RMS perform like overfitting,')

print("Trainning set's RMS result with N-fold cross-validation:")
print("M = 1  ", train_MLE_RMS_N_Fold[0])
print("M = 2  ", train_MLE_RMS_N_Fold[1], '\n')

print("Valid set's RMS result with N-fold cross-validation:")
print("M = 1  ", val_MLE_RMS_N_Fold[0])
print("M = 2  ", val_MLE_RMS_N_Fold[1], '\n')

===Maximum likelihood approach - (c)===

Selecting two different order to compare the behavior of model when performing N-Fold cross validation,
We can found when test validation set in M = 2, the RMS tilts up,
We can conclude that M = 2 may be too much complex for this scenario,
Since its RMS perform like overfitting,
Trainning set's RMS result with N-fold cross-validation:
M = 1   69335.8422335709
M = 2   66816.06577776057 

Valid set's RMS result with N-fold cross-validation:
M = 1   71571.30118693618
M = 2   68421.48684488663 



In [ ]:
# Maximum A Posterior - (a)
print("===Maximum a  Posterior approach - (a)===\n")
print('The key difference between MLP & MAP approach:')
print('If we have prior information, then MAP approach is the way.')
print('Since MLE produces model params. most likely to generate the observed data.')
print('While if no such prior probalilty, then MLE is a reasonable approach.')
print('Since MAP is the model params that is most likely given the observed data.')

===Maximum a  Posterior approach - (a)===

The key difference between MLP & MAP approach:
If we have prior information, then MAP approach is the way.
Since MLE produces model params. most likely to generate the observed data.
While if no such prior probalilty, then MLE is a reasonable approach.
Since MAP is the model params that is most likely given the observed data.


In [ ]:
# Maximum A Posterior - (b)

# Apply a regularizer to (2), the approach will be MAP estimation.

N_Fold = 100
sizeFold = int(trainData.shape[0]/N_Fold)
paramLambda = 0.1

for n in range(N_Fold):
    _val = trainData[sizeFold*n:sizeFold*(n+1), :]
    _train = np.concatenate((trainData[0:sizeFold*n, :], trainData[sizeFold*(n+1):, :]))

    # perform Least Square solution
    # """
    # W = (lambdaI+ X.T*X)^-1 * X.T * Y
    # """ 
    _phiOne, _phiTwo, _ = createPhi(_train[..., :-1])

    _wMAP1 = np.dot(np.dot(
                      np.linalg.pinv((paramLambda* np.eye(_phiOne.shape[1]) + np.dot(_phiOne.T, _phiOne))),
                     _phiOne.T
                  ),
                  _train[:, -1]) # Target
    _wMAP2 = np.dot(np.dot(
                      np.linalg.pinv((paramLambda* np.eye(_phiTwo.shape[1]) + np.dot(_phiTwo.T, _phiTwo))),
                      _phiTwo.T
                  ),
                  _train[:, -1]) # Target
    if n==0:
        _weightMAP1 = _wMAP1
        _weightMAP2 = _wMAP2
    else:
        _weightMAP1 += _wMAP1
        _weightMAP2 += _wMAP2
    
_weightMAP1 = _weightMAP1/N_Fold
_weightMAP2 = _weightMAP2/N_Fold

train_MAP_RMS_N_Fold = [prediction(trainData[..., :-1], trainData[..., -1], 1, _weightMAP1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, _weightMAP2)]
val_MAP_RMS_N_Fold = [prediction(valData[..., :-1], valData[..., -1], 1, _weightMAP1), 
            prediction(valData[..., :-1], valData[..., -1], 2, _weightMAP2)]

In [ ]:
print("===Maximum a  Posterior approach - (b)===\n")

print("Trainning set's RMS result with N-fold cross-validation:")
print("M = 1  ", train_MAP_RMS_N_Fold[0])
print("M = 2  ", train_MAP_RMS_N_Fold[1], '\n')

print("Valid set's RMS result with N-fold cross-validation:")
print("M = 1  ", val_MAP_RMS_N_Fold[0])
print("M = 2  ", val_MAP_RMS_N_Fold[1], '\n')

===Maximum a  Posterior approach - (b)===

Trainning set's RMS result with N-fold cross-validation:
M = 1   69374.94492799495
M = 2   66816.06808266955 

Valid set's RMS result with N-fold cross-validation:
M = 1   71592.30905659722
M = 2   68421.4861225131 



In [ ]:
print("===Maximum a  Posterior approach - (c)===\n")
print('The approach difference is presented above.')
print("But compare with MLE result, show quite similar.")



===Maximum a  Posterior approach - (c)===

The approach difference is presented above.
But compare with MLE result, show quite similar.
